<a href="https://colab.research.google.com/github/RazerRaymond/Pricing-Simulations/blob/main/delta_to_r.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from scipy.optimize import minimize
from scipy.stats import norm
import math

def bsm_delta(S, K, r, sigma, T):
    d1 = (math.log(S / K) + (r + (sigma ** 2) / 2) * T) / (sigma * math.sqrt(T))
    return norm.cdf(d1)

def objective_function(r_guess, delta_observed, S, K, sigma, T):
    delta_calculated = bsm_delta(S, K, r_guess, sigma, T)
    return abs(delta_observed - delta_calculated)

def calculate_risk_free_rate(delta_observed, S, K, sigma, T, r_guess=0.02):
    result = minimize(
        objective_function,
        args=(delta_observed, S, K, sigma, T),
        x0=r_guess,
        bounds=[(0, None)],
        tol=1e-6
    )
    risk_free_rate = result.x[0]
    return risk_free_rate

In [2]:
import numpy as np
from scipy.stats import norm
def BMS_d1(S, K, r, q, sigma, tau):
    ''' Computes d1 for the Black Merton Scholes formula '''
    d1 = 1.0*(np.log(1.0 * S/K) + (r - q + sigma**2/2) * tau) / (sigma * 
np.sqrt(tau))
    return d1
def BMS_d2(S, K, r, q, sigma, tau):
    ''' Computes d2 for the Black Merton Scholes formula '''
    d2 = 1.0*(np.log(1.0 * S/K) + (r - q - sigma**2/2) * tau) / (sigma * 
np.sqrt(tau))
    return d2
def BMS_price(type_option, S, K, r, q, sigma, T, t=0):
    ''' Computes the Black Merton Scholes price for a 'call' or 'put' option '''
    tau = T - t
    d1 = BMS_d1(S, K, r, q, sigma, tau)
    d2 = BMS_d2(S, K, r, q, sigma, tau)
    if type_option == 'call':
        price = S * np.exp(-q * tau) * norm.cdf(d1) - K * np.exp(-r * tau) * norm.cdf(d2)
    elif type_option == 'put':
        price = K * np.exp(-r * tau) * norm.cdf(-d2) - S * np.exp(-q * tau) * norm.cdf(-d1) 
    return price
def BMS_delta(type_option, S, K, r, q, sigma, T, t=0):
    ''' Computes the delta for a call or a put. '''
    tau = T - t
    d1 = BMS_d1(S, K, r, q, sigma, tau)
    if type_option == 'call':
        delta = np.exp(-q * tau) * norm.cdf(d1)
    elif type_option == 'put':
        delta = np.exp(-q * tau) * (norm.cdf(d1) - 1)
    return delta

In [7]:
# problem data
spot = 100
K = 110
r = 0.05
q = 0
sig = 0.22

maturity = 1 # in-years
C = BMS_price('call', spot, K, r, q, sig, maturity)
print('C = ' + str(C))
delta = BMS_delta('call', spot, K, r, q, sig, maturity)
print('del = ' + str(delta))

C = 6.8330484199715045
del = 0.4617780121382925


In [11]:
risk_free_rate = calculate_risk_free_rate(delta, spot, K, sig, maturity)
print("Risk-Free Rate: {}".format(risk_free_rate))

Risk-Free Rate: 0.049999995470113384
